In [ ]:
import os
import sys
import time
import warnings
import pandas as pd
import numpy as np
import QuantLib as ql
from pathlib import Path
import modin.pandas as md
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from matplotlib import cm
from model_settings import ms
pd.set_option("display.float_format", "{:.6f}".format)
warnings.filterwarnings("ignore", ".*defaulting to pandas implementation.*")
notebook_dir = str(Path().resolve())
sys.path.append(str(Path().resolve().parent))
sys.path.append(str(Path().resolve().parent.parent))
from historical_av_key_collector import keys_df, symbol, h5_name
h5_path = os.path.join(notebook_dir,f'alphaVantage {symbol}.h5')
store = pd.HDFStore(h5_path)
store.close()

# data

# calibrations

## calibration testing

## manual recalibration

### ENTRY REPLACEMENT

# checking dates

In [ ]:
keys_df

In [ ]:
keys_df_dates = keys_df.copy().dropna(subset=['calibration_key'])
bar = tqdm(total=(keys_df_dates.shape[0]*keys_df_dates.shape[1]))
while True:
    try:
        with pd.HDFStore(h5_name) as store:
            for i, row in keys_df_dates.iterrows():   
                keys_df_dates.at[i,'calibration_date'] = store[row['calibration_key']]['calculation_date'].drop_duplicates().iloc[0]
                keys_df_dates.at[i,'data_date'] = store[row['raw_data_key']]['date'].drop_duplicates().iloc[0]
                keys_df_dates.at[i,'spot_price_date'] = row['spot_price']
                bar.update(1)
    except OSError as e:
        print('waiting for file')
        time.sleep(2)
    finally:
        store.close()
        bar.close()
keys_df_dates = keys_df_dates.copy()[['calibration_date','data_date','spot_price_date']]

In [ ]:
keys_df_dates

In [ ]:
dates_analysis = keys_df_dates.copy()
def extract_date_from_string(s):
    first_hyphen_pos = s.find('_',0)
    second_slash_pos = s.rfind('/',1)
    return s[first_hyphen_pos+1:second_slash_pos].replace('_','-')

dates_analysis['spot_price_date'] = dates_analysis['spot_price_date'].apply(extract_date_from_string)
print(dates_analysis.dtypes)
dates_analysis

In [ ]:
dates_analysis_problems = dates_analysis[dates_analysis.apply(lambda row: len(set(row)) > 1, axis=1)]
dates_analysis_problems

In [ ]:
problem_date_keys = keys_df.loc[dates_analysis_problems.index,:]
problem_date_keys